# Uncertanty analysis 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl     = px.colors.qualitative.D3
os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 7][["RGIId", "Zone", "ID_basin"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 7]

RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone', 'ID_basin' : 'ID_basin'})

RGI7_ids = RGI7_ids[["RGIId", "Zone", "ID_basin"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

# variables to analize
variable = 'melt_on_glacier'
gdirs = glob("/home/rooda/OGGM_results/new/*", recursive = True)

# Peak water year

In [ ]:
peak_year_dataset = []

for gdir in tqdm(gdirs):
    paths = glob(gdir + "/run_output_*.nc", recursive = True)

    for path in tqdm(paths, leave = False):
        model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc")[variable]
        model_hist   = model_hist.isel(time=slice(0, -1))
        model_future = xr.open_dataset(path)[variable]
        model   = xr.concat([model_hist, model_future], dim = "time")
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        
        model = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())
        model = model.groupby('rgi_id').sum()
        model = model.isel(time=slice(0, -1))
        peak_year = model.rolling(time=10, center=True).mean()
        peak_year = peak_year.idxmax(dim = "time").astype("int16").to_dataframe()
        peak_year = peak_year.transpose()
        
        # data to save
        peak_year.insert(0, 'Outline', os.path.basename(gdir).split("_")[0])
        peak_year.insert(1, 'Climate', os.path.basename(gdir).split("_")[1])
        peak_year.insert(2, 'Volume',  os.path.basename(gdir).split("_")[2])
        peak_year.insert(3, 'GGM',     os.path.basename(path).split("_")[2])
        peak_year.insert(4, 'SSP',     os.path.basename(path).split("_")[3])
        peak_year.insert(5, 'BCM',     os.path.basename(path).split("_")[4][0:3])
        peak_year_dataset.append(peak_year)
        
peak_year_dataset = pd.concat(peak_year_dataset)
peak_year_dataset = peak_year_dataset.reset_index()
peak_year_dataset = peak_year_dataset.drop(["index"], axis = 1)
peak_year_dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/data_basin_peak_water_year.csv")     

# Rate of change 

In [ ]:
rate_change_dataset = []

for gdir in tqdm(gdirs):
    paths = glob(gdir + "/run_output_*.nc", recursive = True)

    for path in tqdm(paths, leave = False): 
        model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc")[variable]
        model_hist   = model_hist.isel(time=slice(0, -1))
        model_future = xr.open_dataset(path)[variable]
        model   = xr.concat([model_hist, model_future], dim = "time")
        
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        model  = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())     
        model = model.groupby('rgi_id').sum()
        model = model.isel(time=slice(0, -1))
        
        # search the year of the peak 
        peak_year = model.rolling(time=10, center=True).mean()
        peak_year = peak_year.idxmax(dim = "time").to_numpy()
        
        # normalize value
        model["melt_on_glacier"] = model/model.max(dim = "time")
        model = model.transpose("rgi_id", "time") # trick to complete loop
        
        rate_change_i = []
        for catchment in range(0, len(model.rgi_id)): # get the trend for each zone
            rate_change = model[catchment].sel(time = slice(peak_year[catchment], peak_year[catchment]+30))
            rate_change = rate_change.polyfit(dim = "time", deg = 1).polyfit_coefficients[0].to_numpy()
            rate_change = rate_change * 100 * 10 # final value: %% per decade
            rate_change_i.append(rate_change)
        
        rate_change = pd.DataFrame(columns =  model.rgi_id.to_numpy())
        rate_change.loc[0] = np.array(rate_change_i)
        
        # data to save
        rate_change.insert(0, 'Outline', os.path.basename(gdir).split("_")[0])
        rate_change.insert(1, 'Climate', os.path.basename(gdir).split("_")[1])
        rate_change.insert(2, 'Volume',  os.path.basename(gdir).split("_")[2])
        rate_change.insert(3, 'GGM',     os.path.basename(path).split("_")[2])
        rate_change.insert(4, 'SSP',     os.path.basename(path).split("_")[3])
        rate_change.insert(5, 'BCM',     os.path.basename(path).split("_")[4][0:3])
        rate_change_dataset.append(rate_change)
        
rate_change_dataset = pd.concat(rate_change_dataset)
rate_change_dataset = rate_change_dataset.reset_index()
rate_change_dataset = rate_change_dataset.drop(["index"], axis = 1)
rate_change_dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/data_basin_rate_change.csv")

# Magnitude

In [ ]:
peak_water_dataset = []

for gdir in tqdm(gdirs):
    paths = glob(gdir + "/run_output_*.nc", recursive = True)

    for path in tqdm(paths, leave = False): 
        model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc")[variable]
        model_hist   = model_hist.isel(time=slice(0, -1))
        model_future = xr.open_dataset(path)[variable]
        model   = xr.concat([model_hist, model_future], dim = "time")
        
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        model  = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())
       
        model   = model.groupby('rgi_id').sum()
        model = model.isel(time=slice(0, -1))
        peak_water = model.max(dim = "time").to_dataframe()
        peak_water = peak_water.transpose() *1e-9
        
        # data to save
        peak_water.insert(0, 'Outline', os.path.basename(gdir).split("_")[0])
        peak_water.insert(1, 'Climate', os.path.basename(gdir).split("_")[1])
        peak_water.insert(2, 'Volume',  os.path.basename(gdir).split("_")[2])
        peak_water.insert(3, 'GGM',     os.path.basename(path).split("_")[2])
        peak_water.insert(4, 'SSP',     os.path.basename(path).split("_")[3])
        peak_water.insert(5, 'BCM',     os.path.basename(path).split("_")[4][0:3])
        peak_water_dataset.append(peak_water)
        
peak_water_dataset = pd.concat(peak_water_dataset)
peak_water_dataset = peak_water_dataset.reset_index()
peak_water_dataset = peak_water_dataset.drop(["index"], axis = 1)
peak_water_dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/data_basin_peak_water_value.csv") 

# Frecuency

In [ ]:
freq_dataset = []

for gdir in tqdm(gdirs):
    paths = glob(gdir + "/run_output_*.nc", recursive = True)

    for path in tqdm(paths, leave = False): 
        model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc")[variable]
        model_hist   = model_hist.isel(time=slice(0, -1))
        model_future = xr.open_dataset(path)[variable]
        model   = xr.concat([model_hist, model_future], dim = "time")
        
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        model  = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())

        model = model.groupby('rgi_id').sum()
        model = model.isel(time=slice(0, -1))
        
        rolling = model.rolling(time=10, center=True).mean()
        frecuency = ((model - rolling)*1e-9)
        frecuency = frecuency.std(dim = "time")
        frecuency = frecuency.to_dataframe().transpose() 
        
        # data to save
        frecuency.insert(0, 'Outline', os.path.basename(gdir).split("_")[0])
        frecuency.insert(1, 'Climate', os.path.basename(gdir).split("_")[1])
        frecuency.insert(2, 'Volume',  os.path.basename(gdir).split("_")[2])
        frecuency.insert(3, 'GGM',     os.path.basename(path).split("_")[2])
        frecuency.insert(4, 'SSP',     os.path.basename(path).split("_")[3])
        frecuency.insert(5, 'BCM',     os.path.basename(path).split("_")[4][0:3])
        freq_dataset.append(frecuency)
        
freq_dataset = pd.concat(freq_dataset)
freq_dataset = freq_dataset.reset_index()
freq_dataset = freq_dataset.drop(["index"], axis = 1)
freq_dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/data_basin_freq.csv") 

# Duration

In [ ]:
variable = "melt_on_glacier_monthly"
duration_dataset = []

for gdir in tqdm(gdirs):
    paths = glob(gdir + "/run_output_*.nc", recursive = True)

    for path in tqdm(paths, leave = False): 
        model_hist   = xr.open_mfdataset(gdir + "/run_outputs_*.nc")[variable]
        model_hist   = model_hist.isel(time=slice(0, -1))
        model_future = xr.open_dataset(path)[variable]
        model   = xr.concat([model_hist, model_future], dim = "time")
        
        ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
        model  = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())
        model = model.groupby('rgi_id').sum()
        model = model.isel(time=slice(0, -1))

        duration = model.mean(dim = "time")*100 / model.mean(dim = "time").sum(dim = "month_2d")
        duration = duration.sel(month_2d = [12,1,2]).sum(dim = "month_2d").to_dataframe().transpose() 

        # data to save
        duration.insert(0, 'Outline', os.path.basename(gdir).split("_")[0])
        duration.insert(1, 'Climate', os.path.basename(gdir).split("_")[1])
        duration.insert(2, 'Volume',  os.path.basename(gdir).split("_")[2])
        duration.insert(3, 'GGM',     os.path.basename(path).split("_")[2])
        duration.insert(4, 'SSP',     os.path.basename(path).split("_")[3])
        duration.insert(5, 'BCM',     os.path.basename(path).split("_")[4][0:3])
        duration_dataset.append(duration)
        
duration_dataset = pd.concat(duration_dataset)
duration_dataset = duration_dataset.reset_index()
duration_dataset = duration_dataset.drop(["index"], axis = 1)
duration_dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/data_basin_duration.csv") 